## Estructura de los Datos

### Cash_Request (CR)

##### CR.Status (23970 registros)

- **money_back**: 16397 registros. El CR fue reembolsado exitosamente.(The CR was successfully reimbursed.)
---
- **active**: 59 registros. Los fondos fueron recibidos en la cuenta del cliente. (Funds were received on the customer account.)
- **direct_debit_sent**: 34 registros. Se envió un débito directo SEPA, pero aún no se confirma el resultado. (We sent/scheduled a SEPA direct debit to charge the customer account. The result of this debit is not yet confirmed)
---
- **rejected**: 6568 registros. El CR necesitó una revisión manual y fue rechazado. (The CR needed a manual review and was rejected)
- **direct_debit_rejected**: 831 registros. El intento de débito directo SEPA falló. (Our last attempt of SEPA direct debit to charge the customer was rejected )
- **transaction_declined**: 48 registros. No se pudo enviar el dinero al cliente. (We failed to send the funds to the customer) 
- **canceled**: 33 registros. El usuario no confirmó el CR en la app, fue cancelado automáticamente. (The user didn't confirm the cash request in-app, we automatically canceled it)

---
**En los datos proporcionados, NO aparecen los valores:** 
- approved : CR is a 'regular' one (= without fees) and was approved either automatically or manually. Funds will be sent aprox. 7 days after the creation
- money_sent : We transferred the fund to the customer account. Will change to active once we detect that the user received the funds (using user's bank history)
- pending : The CR is pending a manual review from an analyst
- waiting_user_confirmation : The user needs to confirm in-app that he want the CR (for legal reasons)
- waiting_reimbursement : We were not able to estimate a date of reimbursement, the user needs to choose one in the app.


##### CR.Transfer Type

- **instant**: El usuario eligió recibir el adelanto instantáneamente. (user choose not received the advance instantly)
- **regular**: El usuario eligió no pagar inmediatamente y esperar la transferencia. (user choose to not pay and wait for the transfer)


##### CR.Recovery Status

- **null**: El CR nunca tuvo un incidente de pago.
- **completed**: El incidente de pago fue resuelto (el CR fue reembolsado).
---
- **pending**: El incidente de pago aún está abierto.
- **pending_direct_debit**: El incidente de pago sigue abierto, pero se ha lanzado un débito directo SEPA.


### Fees (FE)

##### FE.Type

- **instant_payment**: Fees por adelanto instantáneo. (fees for instant cash request (send directly after user's request, through SEPA Instant Payment) )
- **split_payment**: Fees por pago fraccionado (en caso de un incidente). (futures fees for split payment (in case of an incident, we'll soon offer the possibility to our users to reimburse in multiples installements))
- **incident**: Fees por fallos de reembolsos. (fees for failed reimbursement. Created after a failed direct debit)
- **postpone**: Fees por la solicitud de posponer un reembolso. (fees created when a user want to postpone the reimbursment of a CR)

##### FE.Status (= does the fees was successfully charged)

- **accepted**: El fee fue cobrado exitosamente. (fees were successfully charged)
- **confirmed**: El usuario completó una acción que creó un fee. (the user made an action who created a fee. It will normally get charged at the moment of the CR's reimbursement. In some rare cases, postpones are confirmed without being charges due to a commercial offer.)
---
- **rejected**: El último intento de cobrar el fee falló. (the last attempt to charge the fee failed.)
- **cancelled**: El fee fue creado pero cancelado por algún motivo. (fee was created and cancelled for some reasons. It's used to fix issues with fees but it mainly concern postpone fees who failed. We are charging the fees at the moment of the postpone request. If it failed, the postpone is not accepted and the reimbursement date still the same.)

##### FE.Category

- **rejected_direct_debit**: Fees creados cuando el banco del usuario rechaza el primer débito directo. (fees created when user's bank rejects the first direct debit)
- **month_delay_on_payment**: Fees creados cada mes hasta que el incidente se cierre. (fees created every month until the incident is closed)
---
- **null**: No figura a la documentacio

##### FE.paid_at:	

- Timestamp of the fee's payment

##### FE.charge_moment (When the fee will be charge).

- **before**: El fee se cobra en el momento de su creación. (the fee should be charged at the moment of its creation)
- **after**: El fee se cobra cuando el CR es reembolsado. (the fee should be charged at the moment of the CR's reimbursement)

##### FE.total_amount

- Amount of the fee (including VAT)

## Casos

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import itertools # Importando itertools para generar combinaciones de columnas
# Importando la función seasonal_decompose para la descomposición de series temporales
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.patches as mpatches
import payments_manager as pm
#pm.help()
#pm.reset()
#pm.init() #debug=True)
cr_cp = pm.df('cr_cp')
fe_cp = pm.df('fe_cp')
#cr_cp.info()
#fe_cp.info()

df_jo = pm.df('df_jo')
#df_jo.info()
df_jo = pm.sort("df_jo", ["id_cr"]).reset_index()
df_jo = df_jo.drop(columns=['index'])

#df_jo = df_jo.drop(columns=['Mes_created_at'])
df_jo_cp = df_jo.copy()
df_jo_cp['cr_received_date'] = df_jo_cp.cash_request_received_date
#df_jo.info()

In [ ]:
good_cr = ['approved', 'money_sent', 'pending', 'direct_debit_sent', 'active', 'money_back']
good_fe = ['confirmed', 'accepted']
df_jo['good'] = (df_jo['stat_cr'].isin(good_cr)) & (df_jo['stat_fe'].isin(good_fe))


In [5]:
display(df_jo[df_jo.user_id ==2002].reset_index().head(10))

,index,id_cr,id_fe,fe_cr_id,user_id,active,created_at,created_at_fe,amount,fee,...,reco_last_update,updated_at_fe,Mes_created_at,cash_request_received_date,n_fees,n_backs,created_at_slot,created_at_slot_h,created_at_dow,good
0,356,423,0,NaN,2002,1,2020-01-10 10:55:20.757139,NaT,100.0,NaN,...,NaT,NaT,2020-01,2020-01-14,0,1,7,10-Mañana,4,False
1,627,697,0,NaN,2002,1,2020-02-18 16:36:51.249037,NaT,100.0,NaN,...,NaT,NaT,2020-02,2020-02-19,0,2,14,16-Tarde,1,False
2,762,835,0,NaN,2002,1,2020-03-10 07:47:39.337041,NaT,100.0,NaN,...,NaT,NaT,2020-03,2020-03-11,0,3,7,7-Mañana,1,False
3,1098,1172,0,NaN,2002,1,2020-04-14 21:03:09.519326,NaT,100.0,NaN,...,NaT,NaT,2020-04,2020-04-15,0,4,21,21-Noche,1,False
4,1741,1800,0,NaN,2002,1,2020-05-15 04:09:51.091889,NaT,100.0,NaN,...,NaT,NaT,2020-05,2020-05-15,0,5,21,4-Noche,4,False
5,3171,2949,0,NaN,2002,1,2020-06-10 23:34:13.556501,NaT,100.0,NaN,...,NaT,NaT,2020-06,2020-06-13,0,6,21,23-Noche,2,False
6,8381,8177,1977,8177.0,2002,1,2020-07-15 13:17:11.174285,2020-07-25 19:20:00.560197,100.0,5.0,...,NaT,2020-10-13 14:25:04.227093,2020-07,2020-07-18,3,10,7,13-Mañana,2,True
7,8382,8177,1749,8177.0,2002,1,2020-07-15 13:17:11.174285,2020-07-22 01:33:56.400884,100.0,5.0,...,NaT,2020-10-13 14:25:00.882490,2020-07,2020-07-18,1,8,7,13-Mañana,2,True
8,8383,8177,1674,8177.0,2002,1,2020-07-15 13:17:11.174285,2020-07-20 20:43:33.629841,100.0,5.0,...,NaT,2020-10-13 14:25:15.265408,2020-07,2020-07-18,0,7,7,13-Mañana,2,False
9,8384,8177,1839,8177.0,2002,1,2020-07-15 13:17:11.174285,2020-07-23 11:31:44.836318,100.0,5.0,...,NaT,2020-10-13 14:25:02.005106,2020-07,2020-07-18,2,9,7,13-Mañana,2,True


In [4]:
pd.options.display.max_rows = None
df = pm.df('df_jo')
#df.info()

df_cr = pm.df('cr_cp')
df_cr.info()

grouped_counts = df.groupby(
    ['id_cr']).size().reset_index(name='counts').sort_values(
    by=['counts'], ascending=[False]).reset_index(drop=True)
grouped_counts.info()
display(grouped_counts.head(10))


grouped_counts = df.groupby(
    ['id_cr','user_id','stat_fe','stat_cr', 'transfer_type', 'type']).size().reset_index(name='counts').sort_values(
    by=['counts'], ascending=[False]).reset_index(drop=True)
grouped_counts.info()
display(grouped_counts.head(10))

grouped_counts = df.query('stat_fe == "accepted" & stat_cr == "money_back"').groupby(
    ['id_cr','user_id','stat_fe','stat_cr', 'type']).size().reset_index(name='counts').sort_values(
    by=['counts'], ascending=[False]).reset_index(drop=True)
#display(grouped_counts.head(10))


grouped_counts = df.query('stat_fe == "accepted" & stat_cr != "money_back"').groupby(
    ['id_cr','user_id','stat_fe','stat_cr', 'type']).size().reset_index(name='counts').sort_values(
    by=['counts'], ascending=[False]).reset_index(drop=True)
grouped_counts.info()
#display(grouped_counts.head(10))


<class 'pandas.core.frame.DataFrame'>
Index: 23968 entries, 0 to 23969
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype          
---  ------                      --------------  -----          
 0   id                          23968 non-null  int64          
 1   amount                      23968 non-null  float64        
 2   status                      23968 non-null  object         
 3   created_at                  23968 non-null  datetime64[ns] 
 4   updated_at                  23968 non-null  datetime64[ns] 
 5   user_id                     23968 non-null  int64          
 6   moderated_at                16033 non-null  datetime64[ns] 
 7   deleted_account_id          2104 non-null   float64        
 8   reimbursement_date          23968 non-null  datetime64[ns] 
 9   cash_request_received_date  16287 non-null  datetime64[ns] 
 10  money_back_date             16541 non-null  datetime64[ns] 
 11  transfer_type               23968 non-null  ob

,id_cr,counts
0,12225,35
1,5006,28
2,4410,24
3,12452,23
4,11376,21
5,2358,19
6,11746,17
7,15319,16
8,8713,15
9,11968,15


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17444 entries, 0 to 17443
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id_cr          17444 non-null  int64 
 1   user_id        17444 non-null  int64 
 2   stat_fe        17444 non-null  object
 3   stat_cr        17444 non-null  object
 4   transfer_type  17444 non-null  object
 5   type           17444 non-null  object
 6   counts         17444 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 954.1+ KB


,id_cr,user_id,stat_fe,stat_cr,transfer_type,type,counts
0,12225,16391,cancelled,money_back,instant,postpone,33
1,5006,15593,cancelled,money_back,regular,postpone,23
2,4410,99007731,cancelled,money_back,regular,postpone,22
3,12452,23823,cancelled,money_back,instant,postpone,18
4,11376,528,cancelled,money_back,instant,postpone,18
5,2358,11648,cancelled,money_back,regular,postpone,17
6,11746,21934,cancelled,money_back,instant,postpone,15
7,15319,10568,cancelled,active,instant,postpone,15
8,8713,99021706,cancelled,money_back,regular,postpone,14
9,4956,99004558,cancelled,direct_debit_rejected,regular,postpone,14


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298 entries, 0 to 297
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id_cr    298 non-null    int64 
 1   user_id  298 non-null    int64 
 2   stat_fe  298 non-null    object
 3   stat_cr  298 non-null    object
 4   type     298 non-null    object
 5   counts   298 non-null    int64 
dtypes: int64(3), object(3)
memory usage: 14.1+ KB


In [4]:
# Pre-filtrar el dataframe para simplificar cálculos
df_filtered = df_jo[
    (df_jo['stat_fe'] == "accepted") | (df_jo['stat_cr'] == "money_back")
]

# Agregar columnas auxiliares para evitar cálculos repetitivos
df_filtered['accepted_fee'] = df_filtered.apply(
    lambda row: row['fee'] if row['stat_fe'] == "accepted" else 0, axis=1
)
df_filtered['money_back_amount'] = df_filtered.apply(
    lambda row: row['amount'] if row['stat_cr'] == "money_back" else 0, axis=1
)
df_filtered['money_back_id'] = df_filtered.apply(
    lambda row: row['id_cr'] if row['stat_cr'] == "money_back" else None, axis=1
)

# Agrupamiento optimizado
cohort_analysis_2 = df_filtered.groupby(['user_id', 'Mes_created_at'], as_index=False).agg(
    total_paid_fees=('accepted_fee', 'sum'),
    total_paid_cr=('money_back_amount', 'sum'),
    Num_Solicitudes=('money_back_id', 'nunique')
)


/tmp/ipykernel_259809/1991862846.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['accepted_fee'] = df_filtered.apply(
/tmp/ipykernel_259809/1991862846.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['money_back_amount'] = df_filtered.apply(
/tmp/ipykernel_259809/1991862846.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

In [10]:
cohort_analysis_2 = (
    df_jo.query('stat_fe == "accepted" | stat_cr == "money_back"')
        .groupby(['user_id', 'Mes_created_at'], as_index=False)
        .agg(
            total_paid_fees=('fee', lambda x: x[df_jo.loc[x.index, 'stat_fe'] == 'accepted'].sum()),            
            # Contar los valores únicos de 'stat_cr' donde su valor sea 'money_back'
            total_paid_cr=('amount', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].unique().sum()),
            # Contar los valores únicos de 'id_cr' donde 'stat_cr' es igual a 'money_back'
            Num_Solicitudes=('id_cr', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].nunique())
        )
)

In [11]:
display(cohort_analysis_2.head(5))

,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes
0,47,2020-04,0.0,100.0,1
1,47,2020-05,5.0,10.0,1
2,47,2020-08,10.0,10.0,1
3,47,2020-09,5.0,5.0,1
4,47,2020-10,10.0,6.0,2


In [9]:
cohort_analysis_2 = (
    df_jo.query('stat_fe == "accepted" | stat_cr == "money_back"')
        .groupby(['user_id', 'Mes_created_at'], as_index=False)
        .agg(
            # #total_paid_fees=('fee', 'sum'),            
            # #Num_Solicitudes=('id_cr', 'nunique')
            # #Num_Solicitudes=('id_cr', lambda x: x.unique().sum())
            # Num_Solicitudes=('id_cr', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].nunique())

            total_paid_fees=('fee', lambda x: x[df_jo.loc[x.index, 'stat_fe'] == 'accepted'].sum()),            
            # Contar los valores únicos de 'stat_cr' donde su valor sea 'money_back'
            total_paid_cr=('amount', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].unique().sum()),
            # Contar los valores únicos de 'id_cr' donde 'stat_cr' es igual a 'money_back'
            Num_Solicitudes=('id_cr', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].nunique())
        )
)
# Restablecer el índice para un DataFrame limpio (opcional, ya garantizado por as_index=False)
cohort_analysis_2.reset_index(drop=True, inplace=True)

# Calcular el índice como porcentaje entre 'total_paid_fees' y 'total_paid_cr'
cohort_analysis_2['index'] = (
    cohort_analysis_2['total_paid_fees'] / cohort_analysis_2['total_paid_cr'] ) * 100

# Reemplazar valores 'inf' con 0 para manejar divisiones por cero
cohort_analysis_2['index'] = cohort_analysis_2['index'].replace(np.inf, 0)

# Calcular la fecha del último pedido por usuario a partir del DataFrame original
df_jo['created_at'] = pd.to_datetime(df_jo['created_at'])  # Asegurarse de que el formato sea datetime
last_order_per_user = (
    df_jo.groupby('user_id')['created_at']
    .max()  # Obtener la fecha más reciente de pedido para cada usuario
    .dt.to_period('M')  # Convertir a periodo mensual
    .reset_index()  # Restablecer el índice para facilitar el merge
)

# Incorporar la fecha del último pedido en el DataFrame de análisis de cohortes
cohort_analysis_2 = pd.merge(
    cohort_analysis_2,
    last_order_per_user.rename(columns={'created_at': 'last_order'}),
    on='user_id',
    how='left'
)

display(cohort_analysis_2.head(5))

,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,index,last_order
0,47,2020-04,0.0,100.0,1,0.000000,2020-10
1,47,2020-05,5.0,10.0,1,50.000000,2020-10
2,47,2020-08,10.0,10.0,1,100.000000,2020-10
3,47,2020-09,5.0,5.0,1,100.000000,2020-10
4,47,2020-10,10.0,6.0,2,166.666667,2020-10


In [15]:
# Pre-filtrar el dataframe para simplificar cálculos
df_filtered = df_jo[
    (df_jo['stat_fe'] == "accepted") | (df_jo['stat_cr'] == "money_back")
]

# Agregar columnas auxiliares para evitar cálculos repetitivos
df_filtered['accepted_fee'] = df_filtered.apply(
    lambda row: row['fee'] if row['stat_fe'] == "accepted" else 0, axis=1
)
df_filtered['money_back_amount'] = df_filtered.apply(
    lambda row: row['amount'] if row['stat_cr'] == "money_back" else 0, axis=1
)
df_filtered['money_back_id'] = df_filtered.apply(
    lambda row: row['id_cr'] if row['stat_cr'] == "money_back" else None, axis=1
)

# Agrupamiento optimizado
cohort_analysis_2 = df_filtered.groupby(['user_id', 'Mes_created_at'], as_index=False).agg(
    total_paid_fees=('accepted_fee', 'sum'),
    total_paid_cr=('money_back_amount', 'sum'),
    Num_Solicitudes=('money_back_id', 'nunique')
)

# Restablecer el índice para un DataFrame limpio (opcional, ya garantizado por as_index=False)
cohort_analysis_2.reset_index(drop=True, inplace=True)

# Calcular el índice como porcentaje entre 'total_paid_fees' y 'total_paid_cr'
cohort_analysis_2['index'] = (
    cohort_analysis_2['total_paid_fees'] / cohort_analysis_2['total_paid_cr'] ) * 100

# Reemplazar valores 'inf' con 0 para manejar divisiones por cero
cohort_analysis_2['index'] = cohort_analysis_2['index'].replace(np.inf, 0)

# Calcular la fecha del último pedido por usuario a partir del DataFrame original
df_jo['created_at'] = pd.to_datetime(df_jo['created_at'])  # Asegurarse de que el formato sea datetime
last_order_per_user = (
    df_jo.groupby('user_id')['created_at']
    .max()  # Obtener la fecha más reciente de pedido para cada usuario
    .dt.to_period('M')  # Convertir a periodo mensual
    .reset_index()  # Restablecer el índice para facilitar el merge
)

# Incorporar la fecha del último pedido en el DataFrame de análisis de cohortes
cohort_analysis_2 = pd.merge(
    cohort_analysis_2,
    last_order_per_user.rename(columns={'created_at': 'last_order'}),
    on='user_id',
    how='left'
)

display(cohort_analysis_2.head(5))

/tmp/ipykernel_259809/3396134713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['accepted_fee'] = df_filtered.apply(
/tmp/ipykernel_259809/3396134713.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['money_back_amount'] = df_filtered.apply(
/tmp/ipykernel_259809/3396134713.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,index,last_order
0,47,2020-04,0.0,100.0,1,0.000000,2020-10
1,47,2020-05,5.0,50.0,1,10.000000,2020-10
2,47,2020-08,10.0,20.0,1,50.000000,2020-10
3,47,2020-09,5.0,5.0,1,100.000000,2020-10
4,47,2020-10,10.0,6.0,2,166.666667,2020-10


In [16]:
display(cohort_analysis_2[cohort_analysis_2.user_id == 102105])

display(cohort_analysis_2[cohort_analysis_2.user_id == 16391])
display(cohort_analysis_2[cohort_analysis_2.user_id == 2002])
display(cohort_analysis_2[cohort_analysis_2.user_id == 13851])
display(cohort_analysis_2[cohort_analysis_2.user_id == 1987]) #.tail(60))

,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,index,last_order
15482,102105,2020-10,0.0,100.0,1,0.0,2020-10


,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,index,last_order
5421,16391,2020-06,5.0,100.0,1,5.000000,2020-10
5422,16391,2020-08,10.0,3500.0,1,0.285714,2020-10
5423,16391,2020-10,5.0,100.0,1,5.000000,2020-10


,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,index,last_order
747,2002,2020-01,0.0,100.0,1,0.00,2020-10
748,2002,2020-02,0.0,100.0,1,0.00,2020-10
749,2002,2020-03,0.0,100.0,1,0.00,2020-10
750,2002,2020-04,0.0,100.0,1,0.00,2020-10
751,2002,2020-05,0.0,100.0,1,0.00,2020-10
752,2002,2020-06,0.0,100.0,1,0.00,2020-10
753,2002,2020-07,15.0,400.0,1,3.75,2020-10
754,2002,2020-08,0.0,100.0,1,0.00,2020-10
755,2002,2020-09,0.0,100.0,1,0.00,2020-10
756,2002,2020-10,5.0,100.0,1,5.00,2020-10


,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,index,last_order
4640,13851,2020-08,5.0,550.0,2,0.909091,2020-10
4641,13851,2020-10,5.0,100.0,1,5.000000,2020-10


,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,index,last_order
733,1987,2019-12,0.0,100.0,1,0.000000,2020-10
734,1987,2020-01,0.0,100.0,1,0.000000,2020-10
735,1987,2020-02,0.0,100.0,1,0.000000,2020-10
736,1987,2020-03,0.0,100.0,1,0.000000,2020-10
737,1987,2020-04,0.0,100.0,1,0.000000,2020-10
738,1987,2020-05,0.0,100.0,1,0.000000,2020-10
739,1987,2020-06,10.0,200.0,1,5.000000,2020-10
740,1987,2020-08,15.0,300.0,1,5.000000,2020-10
741,1987,2020-10,10.0,300.0,2,3.333333,2020-10


In [7]:
display(cohort_analysis_2[cohort_analysis_2.user_id == 102105])

display(cohort_analysis_2[cohort_analysis_2.user_id == 16391])
display(cohort_analysis_2[cohort_analysis_2.user_id == 2002])
display(cohort_analysis_2[cohort_analysis_2.user_id == 13851])
display(cohort_analysis_2[cohort_analysis_2.user_id == 1987]) #.tail(60))

,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,index,last_order
15482,102105,2020-10,0.0,100.0,1,0.0,2020-10


,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,index,last_order
5421,16391,2020-06,5.0,100.0,1,5.0,2020-10
5422,16391,2020-08,10.0,100.0,1,10.0,2020-10
5423,16391,2020-10,5.0,100.0,1,5.0,2020-10


,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,index,last_order
747,2002,2020-01,0.0,100.0,1,0.0,2020-10
748,2002,2020-02,0.0,100.0,1,0.0,2020-10
749,2002,2020-03,0.0,100.0,1,0.0,2020-10
750,2002,2020-04,0.0,100.0,1,0.0,2020-10
751,2002,2020-05,0.0,100.0,1,0.0,2020-10
752,2002,2020-06,0.0,100.0,1,0.0,2020-10
753,2002,2020-07,15.0,100.0,1,15.0,2020-10
754,2002,2020-08,0.0,100.0,1,0.0,2020-10
755,2002,2020-09,0.0,100.0,1,0.0,2020-10
756,2002,2020-10,5.0,100.0,1,5.0,2020-10


,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,index,last_order
4640,13851,2020-08,5.0,150.0,2,3.333333,2020-10
4641,13851,2020-10,5.0,100.0,1,5.000000,2020-10


,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,index,last_order
733,1987,2019-12,0.0,100.0,1,0.0,2020-10
734,1987,2020-01,0.0,100.0,1,0.0,2020-10
735,1987,2020-02,0.0,100.0,1,0.0,2020-10
736,1987,2020-03,0.0,100.0,1,0.0,2020-10
737,1987,2020-04,0.0,100.0,1,0.0,2020-10
738,1987,2020-05,0.0,100.0,1,0.0,2020-10
739,1987,2020-06,10.0,100.0,1,10.0,2020-10
740,1987,2020-08,15.0,100.0,1,15.0,2020-10
741,1987,2020-10,10.0,100.0,2,10.0,2020-10


In [8]:
pd.options.display.max_rows = None
tops = df_jo[df_jo['stat_cr' ]== 'money_back'].groupby('user_id').agg(fees=('fee','sum'))
tops = df_jo[df_jo['stat_fe'] == 'accepted'  ].groupby('user_id').agg(fees=('fee','sum'))
#display(df_jo[tops])
top_users = tops.sort_values(by='fees', ascending=False).iloc[:10].reset_index()
display(top_users)
#top_users = tops.sort_values(by='fees', ascending=True).iloc[:10]
#display(top_users)

#display(df_jo[top_users])

,user_id,fees
0,17144,75.0
1,12934,55.0
2,4982,35.0
3,99021532,35.0
4,4636,35.0
5,5189,35.0
6,4317,35.0
7,13404,35.0
8,9199,35.0
9,17603,35.0


In [6]:
df = pm.df('df_jo')
timeFormat ='S' #D
df['created_at'] = df['created_at'].dt.to_period(timeFormat) #'Min')
df['created_at_fe'] = df['created_at_fe'].dt.to_period(timeFormat) #'Min')
df['updated_at'] = df['updated_at'].dt.to_period(timeFormat) #'Min')
#df['to_receive_ini'] = df['to_receive_ini'].timedelta(seconds=math.ceil(df['to_receive_ini'].total_seconds()))
df['to_receive_ini'] = pd.to_timedelta(df['to_receive_ini']).round(timeFormat)
df['to_receive_bank'] = pd.to_timedelta(df['to_receive_bank']).round(timeFormat)
df['to_reimbur'] = pd.to_timedelta(df['to_reimbur']).round(timeFormat)
df['to_reimbur_cash'] = pd.to_timedelta(df['to_reimbur_cash']).round(timeFormat)
df['to_end'] = pd.to_timedelta(df['to_end']).round(timeFormat)
df['to_send'] = pd.to_timedelta(df['to_send']).round(timeFormat)
df['money_back_date'] = df['money_back_date'].dt.to_period(timeFormat)
df['send_at'] = df['send_at'].dt.to_period(timeFormat)
df['paid_at'] = df['paid_at'].dt.to_period(timeFormat)
df['moderated_at'] = df['moderated_at'].dt.to_period(timeFormat)
df['from_date'] = df['from_date'].dt.to_period(timeFormat)
df['to_date'] = df['to_date'].dt.to_period(timeFormat)

fields = ['id_cr','created_at','transfer_type','type','stat_cr' ,'amount','fee','n_fees','n_backs','good_user',
          'stat_fe','id_fe','created_at_fe','updated_at_fe','reason','money_back_date', 'reimbursement_date',
          'to_reimbur','from_date','to_date', 'charge_moment' # 'paid_at', 'to_end',, #,'user_id', 'cr_received_date','recovery_status'
          #'to_receive_ini','to_receive_bank' #,'to_reimbur_cash', 'updated_at', 'to_send','send_at','moderated_at'
]

user_id = 2002# 16391 # 2002, 1987, 13851, 16391, 102105
display(cohort_analysis_2[cohort_analysis_2.user_id == user_id])

#print("Casos segun Cash Request ID")
pd.options.display.max_columns = None
for id in ([-8177]): # 16391 20108, 20104, 20112,
    df_t = df[df['id_cr'] == id].sort_values(['created_at','created_at_fe']).reset_index()
    print(f"Cash Request ID: {id}")
    display(df_t[fields])

user_ids = [user_id] 
pd.options.display.max_columns = None
#print("Casos segun Cash User ID")
for id in (user_ids):
    df_t = df[(df['user_id'] == id)]#.reset_index()
    df_t = df_t[df_t['stat_cr'] == 'money_back']
    df_t = df_t[df_t['stat_fe'] == 'accepted']
    
    df_t = df_t.sort_values(['created_at','created_at_fe']).reset_index(drop=True)
    #df_t.set_index('id_cr', inplace=True)
    print(f"Only money_back - user_id {id}")
    display(df_t[fields])
    df_t = df[(df['user_id'] == id) ].sort_values(['created_at','created_at_fe']).reset_index(drop=True)

    print(f"user_id {id}")
    display(df_t[fields])

#user_ids = [13851] [2002] , 1987, 1946, 90, 526, 12934] #, 12274 54879 12441, 13851, 16391, 430,  63894,18730,10116,21465, 99000262]
# vips 12934 526
# 90 Este se esta gestionando mal: todos instant, con demoras y sin gestion por 
# 1946 Parece un ejemplo de buena gestion, al final tiene un instant y se le ha dado margen el las demoras.
# 1987 Parece un ejemplo de buen usuario, se pasa a instant para siempre.


,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes
747,2002,2020-01,0.0,100.0,1
748,2002,2020-02,0.0,100.0,1
749,2002,2020-03,0.0,100.0,1
750,2002,2020-04,0.0,100.0,1
751,2002,2020-05,0.0,100.0,1
752,2002,2020-06,0.0,100.0,1
753,2002,2020-07,15.0,400.0,1
754,2002,2020-08,0.0,100.0,1
755,2002,2020-09,0.0,100.0,1
756,2002,2020-10,5.0,100.0,1


Cash Request ID: -8177


,id_cr,created_at,transfer_type,type,stat_cr,amount,fee,n_fees,n_backs,good_user,stat_fe,id_fe,created_at_fe,updated_at_fe,reason,money_back_date,reimbursement_date,to_reimbur,from_date,to_date,charge_moment


Only money_back - user_id 2002


,id_cr,created_at,transfer_type,type,stat_cr,amount,fee,n_fees,n_backs,good_user,stat_fe,id_fe,created_at_fe,updated_at_fe,reason,money_back_date,reimbursement_date,to_reimbur,from_date,to_date,charge_moment
0,8177,2020-07-15 13:17:11,regular,postpone,money_back,100.0,5.0,1,8,True,accepted,1749,2020-07-22 01:33:56,2020-10-13 14:25:00.882490,Postpone Cash Request 8177,2020-08-03 09:01:41,2020-08-05 01:33:48,20 days 12:16:37,2020-08-04 22:00:00,2020-08-15 01:33:48,before
1,8177,2020-07-15 13:17:11,regular,postpone,money_back,100.0,5.0,2,9,True,accepted,1839,2020-07-23 11:31:44,2020-10-13 14:25:02.005106,Postpone Cash Request 8177,2020-08-03 09:01:41,2020-08-05 01:33:48,20 days 12:16:37,2020-08-15 01:33:48,2020-08-30 01:33:48,before
2,8177,2020-07-15 13:17:11,regular,postpone,money_back,100.0,5.0,3,10,True,accepted,1977,2020-07-25 19:20:00,2020-10-13 14:25:04.227093,Postpone Cash Request 8177,2020-08-03 09:01:41,2020-08-05 01:33:48,20 days 12:16:37,2020-08-30 01:33:48,2020-09-14 01:33:48,before
3,24248,2020-10-25 07:41:00,instant,instant_payment,money_back,100.0,5.0,4,13,True,accepted,17289,2020-10-25 07:41:49,2020-10-25 07:41:49.198245,Instant Payment Cash Request 24248,2020-11-09 19:11:48,2020-11-09 22:00:00,15 days 14:19:00,NaT,NaT,after


user_id 2002


,id_cr,created_at,transfer_type,type,stat_cr,amount,fee,n_fees,n_backs,good_user,stat_fe,id_fe,created_at_fe,updated_at_fe,reason,money_back_date,reimbursement_date,to_reimbur,from_date,to_date,charge_moment
0,423,2020-01-10 10:55:20,regular,nice,money_back,100.0,NaN,0,1,False,NaN,0,NaT,NaT,NaN,2020-02-06 23:00:00,2020-02-06 23:00:00,27 days 12:04:39,NaT,NaT,NaN
1,697,2020-02-18 16:36:51,regular,nice,money_back,100.0,NaN,0,2,False,NaN,0,NaT,NaT,NaN,2020-02-27 23:00:00,2020-02-27 23:00:00,9 days 06:23:09,NaT,NaT,NaN
2,835,2020-03-10 07:47:39,regular,nice,money_back,100.0,NaN,0,3,False,NaN,0,NaT,NaT,NaN,2020-04-14 20:25:59,2020-04-05 22:00:00,26 days 14:12:21,NaT,NaT,NaN
3,1172,2020-04-14 21:03:09,regular,nice,money_back,100.0,NaN,0,4,False,NaN,0,NaT,NaT,NaN,2020-05-14 21:05:09,2020-05-05 22:00:00,21 days 00:56:50,NaT,NaT,NaN
4,1800,2020-05-15 04:09:51,regular,nice,money_back,100.0,NaN,0,5,False,NaN,0,NaT,NaT,NaN,2020-06-10 22:17:53,2020-06-06 22:00:00,22 days 17:50:09,NaT,NaT,NaN
5,2949,2020-06-10 23:34:13,regular,nice,money_back,100.0,NaN,0,6,False,NaN,0,NaT,NaT,NaN,2020-07-07 22:00:00,2020-07-06 22:00:00,25 days 22:25:46,NaT,NaT,NaN
6,8177,2020-07-15 13:17:11,regular,postpone,money_back,100.0,5.0,0,7,False,cancelled,1674,2020-07-20 20:43:33,2020-10-13 14:25:15.265408,Postpone Cash Request 8177,2020-08-03 09:01:41,2020-08-05 01:33:48,20 days 12:16:37,2020-08-04 22:00:00,2020-08-19 22:00:00,after
7,8177,2020-07-15 13:17:11,regular,postpone,money_back,100.0,5.0,1,8,True,accepted,1749,2020-07-22 01:33:56,2020-10-13 14:25:00.882490,Postpone Cash Request 8177,2020-08-03 09:01:41,2020-08-05 01:33:48,20 days 12:16:37,2020-08-04 22:00:00,2020-08-15 01:33:48,before
8,8177,2020-07-15 13:17:11,regular,postpone,money_back,100.0,5.0,2,9,True,accepted,1839,2020-07-23 11:31:44,2020-10-13 14:25:02.005106,Postpone Cash Request 8177,2020-08-03 09:01:41,2020-08-05 01:33:48,20 days 12:16:37,2020-08-15 01:33:48,2020-08-30 01:33:48,before
9,8177,2020-07-15 13:17:11,regular,postpone,money_back,100.0,5.0,3,10,True,accepted,1977,2020-07-25 19:20:00,2020-10-13 14:25:04.227093,Postpone Cash Request 8177,2020-08-03 09:01:41,2020-08-05 01:33:48,20 days 12:16:37,2020-08-30 01:33:48,2020-09-14 01:33:48,before


In [8]:
cohort_analysis_2 = (
    df_jo.groupby(['user_id', 'Mes_created_at'], as_index=False)
    .agg(
        # Sumar los valores de 'fee' donde 'stat_fe' es igual a 'accepted'
        total_paid_fees=('fee', lambda x: x[df_jo.loc[x.index, 'stat_fe'] == 'accepted'].sum()),
        
        # Contar los valores únicos de 'stat_cr' donde su valor sea 'money_back'
        total_paid_cr=('amount', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].unique().sum()),

        # Contar los valores únicos de 'id_cr' donde 'stat_cr' es igual a 'money_back'
        Num_Solicitudes=('id_cr', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].nunique())
    )
)

In [9]:
display(cohort_analysis_2[cohort_analysis_2.user_id == 2002]) ## 13851]) #.tail(60))

,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes
996,2002,2020-01,0.0,100.0,1
997,2002,2020-02,0.0,100.0,1
998,2002,2020-03,0.0,100.0,1
999,2002,2020-04,0.0,100.0,1
1000,2002,2020-05,0.0,100.0,1
1001,2002,2020-06,0.0,100.0,1
1002,2002,2020-07,15.0,100.0,1
1003,2002,2020-08,0.0,100.0,1
1004,2002,2020-09,0.0,100.0,1
1005,2002,2020-10,5.0,100.0,1


In [10]:
# Filtrar només les files necessàries per les operacions
#filtered_df = df_jo[df_jo['stat_fe'] == 'accepted']
filtered_df = df_jo.copy()

# Agregar les dades agrupades
cohort_analysis_2 = (
    filtered_df.groupby(['user_id', 'Mes_created_at'], as_index=False)
    .agg(
        #total_paid_fees=('fee', 'sum'),
        total_paid_fees=('fee', lambda x: x[df_jo.loc[x.index, 'stat_fe'] == 'accepted'].sum()),
        total_paid_cr=('amount', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].unique().sum()),
        Num_Solicitudes=('id_cr', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].nunique()),
        transfer_type=('transfer_type', lambda x: ', '.join(x.unique()))  # Llista valors únics
    )
)

In [11]:
display(cohort_analysis_2[cohort_analysis_2.user_id == 13851]) #.tail(60))

,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,transfer_type
5689,13851,2020-06,0.0,0.0,0,regular
5690,13851,2020-07,0.0,0.0,0,regular
5691,13851,2020-08,5.0,150.0,2,"regular, instant"
5692,13851,2020-10,5.0,100.0,1,instant


In [12]:
#cohort_analysis_3 = filtered_df.groupby(['user_id', 'Mes_created_at'])
#display(cohort_analysis_3)

In [13]:
# Restablecer el índice para un DataFrame limpio (opcional, ya garantizado por as_index=False)
cohort_analysis_2.reset_index(drop=True, inplace=True)

# Calcular el índice como porcentaje entre 'total_paid_fees' y 'total_paid_cr'
cohort_analysis_2['index'] = (
    cohort_analysis_2['total_paid_fees'] / cohort_analysis_2['total_paid_cr']
) * 100

# Reemplazar valores 'inf' con 0 para manejar divisiones por cero
cohort_analysis_2['index'] = cohort_analysis_2['index'].replace(np.inf, 0)

# Calcular la fecha del último pedido por usuario a partir del DataFrame original
df_jo['created_at'] = pd.to_datetime(df_jo['created_at'])  # Asegurarse de que el formato sea datetime
last_order_per_user = (
    df_jo.groupby('user_id')['created_at']
    .max()  # Obtener la fecha más reciente de pedido para cada usuario
    .dt.to_period('M')  # Convertir a periodo mensual
    .reset_index()  # Restablecer el índice para facilitar el merge
)

# Incorporar la fecha del último pedido en el DataFrame de análisis de cohortes
cohort_analysis_2 = pd.merge(
    cohort_analysis_2,
    last_order_per_user.rename(columns={'created_at': 'last_order'}),
    on='user_id',
    how='left'
)

# Resultado final del DataFrame de análisis de cohortes
#cohort_analysis_2
display(cohort_analysis_2[cohort_analysis_2.user_id == 13851]) #.tail(60))

,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,transfer_type,index,last_order
5689,13851,2020-06,0.0,0.0,0,regular,NaN,2020-10
5690,13851,2020-07,0.0,0.0,0,regular,NaN,2020-10
5691,13851,2020-08,5.0,150.0,2,"regular, instant",3.333333,2020-10
5692,13851,2020-10,5.0,100.0,1,instant,5.000000,2020-10


In [14]:
#cohort_analysis_2.info
#cohort_analysis_2.count
display(cohort_analysis_2[cohort_analysis_2.user_id == 13851]) #.tail(60))


,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,transfer_type,index,last_order
5689,13851,2020-06,0.0,0.0,0,regular,NaN,2020-10
5690,13851,2020-07,0.0,0.0,0,regular,NaN,2020-10
5691,13851,2020-08,5.0,150.0,2,"regular, instant",3.333333,2020-10
5692,13851,2020-10,5.0,100.0,1,instant,5.000000,2020-10


In [15]:
# Crear columna 'Cohorte' que indica el mes de la primera solicitud de cada cliente
df_jo['Cohorte'] = df_jo.groupby('user_id')['created_at'].transform('min').dt.to_period('M')
#df_jo.head(10)

In [16]:
# Agrupar los datos por cohorte y mes para calcular las métricas mensuales
cohort_analysis = df_jo.groupby(['Cohorte', 'Mes_created_at']).agg(
    Num_Clientes=('user_id', 'nunique'),  # Número de clientes únicos por cohorte y mes
    Num_Solicitudes=('id_cr', 'nunique'),  # Total de solicitudes únicas por cohorte y mes
    Solicitudes_Instant=('transfer_type', lambda x: (x == 'instant').sum()),  # Solicitudes tipo 'instant' por cohorte y mes
    Solicitudes_Regular=('transfer_type', lambda x: (x == 'regular').sum()),  # Solicitudes tipo 'regular' por cohorte y mes
    Clientes_con_fees=('fee', lambda x: (x > 0).sum()),  # Clientes con fees por cohorte y mes
    CR=('id_cr', lambda ids: df_jo[df_jo['id_cr'].isin(ids) & df_jo['cash_request_received_date'].notnull()]['amount'].sum()),

    Fees=('fee', 'sum'),  # Total de fees por cohorte y mes
    # Contar las líneas con 'stat_fe' == 'accepted'
    paid_fees=('stat_fe', lambda x: (x == 'accepted').sum()),  
    # Contar las líneas con 'stat_fe' != 'accepted'
    non_paid_fees=('stat_fe', lambda x: (x != 'accepted').sum()),
    # Sumar 'total_amount' cuando 'stat_fe' == 'accepted'
    total_paid_fees=('fee', lambda x: x[df_jo.loc[x.index, 'stat_fe'] == 'accepted'].sum()),
    # Sumar 'total_amount' cuando 'stat_fe' != 'accepted'
    total_non_paid_fees=('fee', lambda x: x[df_jo.loc[x.index, 'stat_fe'] != 'accepted'].sum()),
     # Contar las líneas con 'stat_cr' == 'money_back'
    paid_cr=('stat_cr', lambda x: (x == 'money_back').sum()),  
    # Contar las líneas con 'stat_fe' != 'accepted'
    non_paid_cr=('stat_cr', lambda x: (x != 'money_back').sum()),
    # Sumar 'total_amount' cuando 'stat_fe' == 'accepted'
    total_paid_cr=('amount', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].sum()),
    # Sumar 'total_amount' cuando 'stat_fe' != 'accepted'
    total_non_paid_cr=('amount', lambda x: x[df_jo.loc[x.index, 'stat_cr'] != 'money_back'].sum()),
    # Métrica de "credit lapse" (diferencia entre reimbursement_date y created_at)
    credit_lapse=('reimbursement_date', lambda x: (x - df_jo.loc[x.index, 'created_at']).dt.days.mean()),
    # Métrica de "credit payment lapse" (diferencia entre money_back_date y reimbursement_date)
    credit_payment_lapse=('money_back_date', lambda x: (x - df_jo.loc[x.index, 'reimbursement_date']).dt.days.mean())

).reset_index()

# Calcular los porcentajes de non_paid_fees
cohort_analysis['paid_fees_percent_qty'] = (cohort_analysis['paid_fees'] / (cohort_analysis['paid_fees'] + cohort_analysis['non_paid_fees']))
cohort_analysis['paid_fees_percent_amount'] = (cohort_analysis['total_paid_fees'] / cohort_analysis['Fees'])   # % en monto
cohort_analysis['paid_cr_percent_qty'] = (cohort_analysis['paid_cr'] / (cohort_analysis['paid_cr'] + cohort_analysis['non_paid_cr']))
cohort_analysis['paid_cr_percent_amount'] = (cohort_analysis['total_paid_cr'] / cohort_analysis['CR'])   # % en monto
cohort_analysis['index'] = (cohort_analysis['total_paid_fees'] / cohort_analysis['total_paid_cr'])   # % index

# Eliminar las columnas de pago y no pago
cohort_sizes_monthly = cohort_analysis.drop(
    ['paid_fees', 'non_paid_fees', 'total_paid_fees', 'total_non_paid_fees','paid_cr', 'non_paid_cr', 'total_paid_cr', 'total_non_paid_cr'], axis=1, errors='ignore')

# Mostrar los resultados de las cohortes por mes
#cohort_analysis

In [17]:
cohort_analysis
#display(cohort_analysis[cohort_analysis.user_id == 2002])

,Cohorte,Mes_created_at,Num_Clientes,Num_Solicitudes,Solicitudes_Instant,Solicitudes_Regular,Clientes_con_fees,CR,Fees,paid_fees,non_paid_fees,total_paid_fees,total_non_paid_fees,paid_cr,non_paid_cr,total_paid_cr,total_non_paid_cr,credit_lapse,credit_payment_lapse,paid_fees_percent_qty,paid_fees_percent_amount,paid_cr_percent_qty,paid_cr_percent_amount,index
0,2019-11,2019-11,1,1,0,1,0,0.0,0.0,0,1,0.0,0.0,0,1,0.0,1.0,16.000000,NaN,0.000000,NaN,0.000000,NaN,NaN
1,2019-11,2019-12,1,1,0,1,0,0.0,0.0,0,1,0.0,0.0,0,1,0.0,100.0,6.000000,NaN,0.000000,NaN,0.000000,NaN,NaN
2,2019-11,2020-04,1,1,0,1,0,100.0,0.0,0,1,0.0,0.0,1,0,100.0,0.0,14.000000,8.000000,0.000000,NaN,1.000000,1.000000,0.000000
3,2019-11,2020-05,1,1,0,5,5,50.0,25.0,1,4,5.0,20.0,5,0,50.0,0.0,125.000000,-45.000000,0.200000,0.200000,1.000000,1.000000,0.100000
4,2019-11,2020-08,1,1,2,0,2,20.0,10.0,2,0,10.0,0.0,2,0,20.0,0.0,55.000000,-8.000000,1.000000,1.000000,1.000000,1.000000,0.500000
5,2019-11,2020-09,1,1,1,0,1,5.0,5.0,1,0,5.0,0.0,1,0,5.0,0.0,23.000000,-8.000000,1.000000,1.000000,1.000000,1.000000,1.000000
6,2019-11,2020-10,1,2,2,0,2,6.0,10.0,2,0,10.0,0.0,2,0,6.0,0.0,29.500000,-18.500000,1.000000,1.000000,1.000000,1.000000,1.666667
7,2019-12,2019-12,256,288,0,288,0,16349.0,0.0,0,288,0.0,0.0,154,134,14490.0,12707.0,26.180556,23.950311,0.000000,NaN,0.534722,0.886293,0.000000
8,2019-12,2020-01,89,100,0,100,0,5713.0,0.0,0,100,0.0,0.0,58,42,5513.0,4030.0,38.390000,28.137931,0.000000,NaN,0.580000,0.964992,0.000000
9,2019-12,2020-02,55,58,0,58,0,4100.0,0.0,0,58,0.0,0.0,37,21,3600.0,1781.0,22.568966,10.243243,0.000000,NaN,0.637931,0.878049,0.000000


In [18]:
df = pm.df('df_jo')
#df.info()

cr_id = ['id_cr','id_fe','user_id', 'created_at','created_at_fe','amount','fee','stat_cr','stat_fe','transfer_type','type',
            'to_receive_ini', 'to_receive_bank','to_reimbur','to_reimbur_cash','to_end','to_send',
             'send_at', 'cr_received_date', 'money_back_date', 'reimbursement_date',
            'paid_at','charge_moment','moderated_at','reason','category','from_date','to_date', 'recovery_status'] # ,'id_y','from_date','to_date','reason','recovery_status', 'cash_request_id'

print("Casos segun Cash Request ID")
pd.options.display.max_columns = None
for id in ([-18264]): # 16391 20108, 20104, 20112,
    df_t = df[df['id_cr'] == id].reset_index()
    print(f"Cash Request ID: {id}")
    display(df_t[cr_id].sort_values('created_at'))

user_ids = [54879, 9900458, 2002, 1987, 1946, 90, 526, 12934] #, 12274 54879 12441, 13851, 16391, 430,  63894,18730,10116,21465, 99000262]
# vips 12934 526
# 90 Este se esta gestionando mal: todos instant, con demoras y sin gestion por 
# 1946 Parece un ejemplo de buena gestion, al final tiene un instant y se le ha dado margen el las demoras.
# 1987 Parece un ejemplo de buen usuario, se pasa a instant para siempre.

pd.options.display.max_columns = None
print("Casos segun Cash User ID")
for id in (user_ids):
    #df_t = df[df['user_id'] == id].reset_index()
    df_t = df[(df['user_id'] == id) & (df['stat_cr'] == 'money_back')].reset_index()
    print(f"User ID: {id}")
    display(df_t[cr_id].sort_values('created_at').reset_index())

Casos segun Cash Request ID
Cash Request ID: -18264


,id_cr,id_fe,user_id,created_at,created_at_fe,amount,fee,stat_cr,stat_fe,transfer_type,type,to_receive_ini,to_receive_bank,to_reimbur,to_reimbur_cash,to_end,to_send,send_at,cr_received_date,money_back_date,reimbursement_date,paid_at,charge_moment,moderated_at,reason,category,from_date,to_date,recovery_status


Casos segun Cash User ID
User ID: 54879


,index,id_cr,id_fe,user_id,created_at,created_at_fe,amount,fee,stat_cr,stat_fe,transfer_type,type,to_receive_ini,to_receive_bank,to_reimbur,to_reimbur_cash,to_end,to_send,send_at,cr_received_date,money_back_date,reimbursement_date,paid_at,charge_moment,moderated_at,reason,category,from_date,to_date,recovery_status


User ID: 9900458


...

,index,id_cr,id_fe,user_id,created_at,created_at_fe,amount,fee,stat_cr,stat_fe,transfer_type,type,to_receive_ini,to_receive_bank,to_reimbur,to_reimbur_cash,to_end,to_send,send_at,cr_received_date,money_back_date,reimbursement_date,paid_at,charge_moment,moderated_at,reason,category,from_date,to_date,recovery_status
0,3,314,0,90,2019-12-26 19:20:47.806186,NaT,100.0,NaN,money_back,NaN,regular,nice,4 days 04:39:12.193814,NaT,13 days 03:39:12.193814,NaT,NaT,NaT,NaT,2019-12-31,2020-01-08 23:00:00.000000,2020-01-08 23:00:00.000000,NaT,NaN,2019-12-27 14:06:27.008412,NaN,NaN,NaT,NaT,nice
1,4,667,0,90,2020-02-11 19:04:59.715551,NaT,100.0,NaN,money_back,NaN,regular,nice,0 days 04:55:00.284449,NaT,19 days 03:55:00.284449,NaT,-9 days +01:29:24.798386,NaT,NaT,2020-02-12,2020-03-10 21:30:35.201614,2020-03-01 23:00:00.000000,NaT,NaN,2020-02-12 11:17:31.780167,NaN,NaN,NaT,NaT,nice
2,1,893,0,90,2020-03-16 14:56:43.473601,NaT,100.0,NaN,money_back,NaN,regular,nice,1 days 09:03:16.526399,NaT,21 days 07:03:16.526399,NaT,-9 days +01:08:59.185284,NaT,NaT,2020-03-18,2020-04-15 20:51:00.814716,2020-04-06 22:00:00.000000,NaT,NaN,2020-03-16 15:11:42.665961,NaN,NaN,NaT,NaT,nice
3,0,2155,0,90,2020-05-25 12:48:13.019894,NaT,100.0,NaN,money_back,NaN,regular,nice,2 days 11:11:46.980106,NaT,11 days 09:11:46.980106,NaT,-7 days +23:22:01.202196,NaT,NaT,2020-05-28,2020-06-11 22:37:58.797804,2020-06-05 22:00:00.000000,NaT,NaN,2020-05-25 13:20:08.786743,NaN,NaN,NaT,NaT,nice
4,2,3343,0,90,2020-06-15 10:39:34.100529,NaT,100.0,NaN,money_back,NaN,regular,nice,2 days 13:20:25.899471,NaT,9 days 23:59:59.991291,NaT,-2 days +07:14:00.005528,NaT,NaT,2020-06-18,2020-06-27 03:25:34.086292,2020-06-25 10:39:34.091820,NaT,NaN,2020-06-15 11:52:28.283050,NaN,NaN,NaT,NaT,nice
5,5,5608,0,90,2020-06-27 14:08:39.137822,NaT,100.0,NaN,money_back,NaN,regular,nice,2 days 09:51:20.862178,NaT,11 days 09:42:20.862178,NaT,-1 days +01:51:00,NaT,NaT,2020-06-30,2020-07-09 22:00:00.000000,2020-07-08 23:51:00.000000,NaT,NaN,2020-06-27 17:43:48.707336,NaN,NaN,NaT,NaT,nice
6,8,9206,0,90,2020-07-21 08:16:11.513030,NaT,100.0,NaN,money_back,NaN,regular,nice,1 days 15:43:48.486970,-6 days +15:43:48.487391,9 days 13:43:48.486970,2 days 13:43:48.487391,0 days 00:00:00,6 days 23:59:59.999579,2020-07-28 08:16:11.512609,2020-07-23,2020-07-30 22:00:00.000000,2020-07-30 22:00:00.000000,NaT,NaN,2020-07-21 13:49:36.592679,NaN,NaN,NaT,NaT,nice
7,6,11384,0,90,2020-08-07 14:50:30.098500,NaT,100.0,NaN,money_back,NaN,regular,nice,10 days 09:09:29.901500,3 days 09:09:29.902154,21 days 07:09:29.901500,14 days 07:09:29.902154,-2 days +00:00:00,6 days 23:59:59.999346,2020-08-14 14:50:30.097846,2020-08-18,2020-08-30 22:00:00.000000,2020-08-28 22:00:00.000000,NaT,NaN,2020-08-07 15:58:06.719912,NaN,NaN,NaT,NaT,nice
8,7,14367,0,90,2020-09-03 12:54:09.185674,NaT,100.0,NaN,money_back,NaN,regular,nice,8 days 11:05:50.814326,1 days 11:05:50.883830,28 days 09:05:50.814326,21 days 09:05:50.883830,-1 days +10:33:46.582802,6 days 23:59:59.930496,2020-09-10 12:54:09.116170,2020-09-12,2020-10-02 11:26:13.417198,2020-10-01 22:00:00.000000,NaT,NaN,NaT,NaN,NaN,NaT,NaT,nice
9,9,19294,0,90,2020-10-05 15:34:36.390378,NaT,100.0,NaN,money_back,NaN,regular,nice,8 days 08:25:23.609622,1 days 08:25:23.654154,28 days 06:25:23.609622,21 days 06:25:23.654154,0 days 02:22:19.535709,6 days 23:59:59.955468,2020-10-12 15:34:36.345846,2020-10-14,2020-11-02 19:37:40.464291,2020-11-02 22:00:00.000000,NaT,NaN,2020-10-05 15:56:11.690041,NaN,NaN,NaT,NaT,nice


User ID: 526


,index,id_cr,id_fe,user_id,created_at,created_at_fe,amount,fee,stat_cr,stat_fe,transfer_type,type,to_receive_ini,to_receive_bank,to_reimbur,to_reimbur_cash,to_end,to_send,send_at,cr_received_date,money_back_date,reimbursement_date,paid_at,charge_moment,moderated_at,reason,category,from_date,to_date,recovery_status
0,3,131,0,526,2019-12-12 20:59:48.763947,NaT,90.0,NaN,money_back,NaN,regular,nice,3 days 03:00:11.236053,NaT,14 days 02:00:11.236053,NaT,NaT,NaT,NaT,2019-12-16,2019-12-26 23:00:00.000000,2019-12-26 23:00:00,NaT,NaN,2019-12-13 16:32:34.692499,NaN,NaN,NaT,NaT,nice
1,0,614,0,526,2020-02-05 17:54:50.727851,NaT,50.0,NaN,money_back,NaN,regular,nice,2 days 06:05:09.272149,NaT,22 days 05:05:09.272149,NaT,NaT,NaT,NaT,2020-02-08,2020-02-27 23:00:00.000000,2020-02-27 23:00:00,NaT,NaN,2020-02-06 12:48:34.924114,NaN,NaN,NaT,NaT,nice
2,2,824,0,526,2020-03-08 14:01:05.146330,NaT,60.0,NaN,money_back,NaN,regular,nice,1 days 09:58:54.853670,NaT,7 days 08:58:54.853670,NaT,-10 days +23:19:15.010974,NaT,NaT,2020-03-10,2020-03-24 23:40:44.989026,2020-03-15 23:00:00,NaT,NaN,2020-03-08 14:57:10.702615,NaN,NaN,NaT,NaT,nice
3,5,985,0,526,2020-03-25 02:27:41.780578,NaT,90.0,NaN,money_back,NaN,regular,nice,1 days 21:32:18.219422,NaT,19 days 19:32:18.219422,NaT,-9 days +01:09:05.843302,NaT,NaT,2020-03-27,2020-04-22 20:50:54.156698,2020-04-13 22:00:00,NaT,NaN,2020-03-25 11:00:18.003849,NaN,NaN,NaT,NaT,nice
4,4,1348,0,526,2020-04-23 18:54:05.810157,NaT,80.0,NaN,money_back,NaN,regular,nice,3 days 05:05:54.189843,NaT,17 days 03:05:54.189843,NaT,-12 days +08:11:47.735425,NaT,NaT,2020-04-27,2020-05-22 13:48:12.264575,2020-05-10 22:00:00,NaT,NaN,2020-04-24 08:41:52.485739,NaN,NaN,NaT,NaT,completed
5,6,3478,0,526,2020-06-16 12:38:38.140763,NaT,80.0,NaN,money_back,NaN,regular,nice,1 days 11:21:21.859237,NaT,12 days 09:21:21.859237,NaT,-2 days +13:13:24.063854,NaT,NaT,2020-06-18,2020-06-30 08:46:35.936146,2020-06-28 22:00:00,NaT,NaN,2020-06-16 13:26:21.815821,NaN,NaN,NaT,NaT,nice
6,7,7250,1904,526,2020-07-08 22:46:36.854214,2020-07-24 17:07:26.992132,90.0,5.0,money_back,cancelled,regular,postpone,1 days 01:13:23.145786,NaT,20 days 23:13:23.145786,NaT,-13 days +00:00:00,NaT,NaT,2020-07-10,2020-08-11 22:00:00.000000,2020-07-29 22:00:00,NaT,before,2020-07-09 08:56:37.509660,Postpone Cash Request 7250,nice,2020-07-29 22:00:00,2020-08-10 22:00:00,nice
7,8,7250,2102,526,2020-07-08 22:46:36.854214,2020-07-27 12:52:44.616105,90.0,5.0,money_back,cancelled,regular,postpone,1 days 01:13:23.145786,NaT,20 days 23:13:23.145786,NaT,-13 days +00:00:00,NaT,NaT,2020-07-10,2020-08-11 22:00:00.000000,2020-07-29 22:00:00,NaT,before,2020-07-09 08:56:37.509660,Postpone Cash Request 7250,nice,2020-07-29 22:00:00,2020-08-09 22:00:00,nice
8,9,12484,4815,526,2020-08-14 12:28:42.222162,2020-08-22 15:08:31.470344,90.0,5.0,money_back,cancelled,instant,postpone,3 days 11:31:17.777838,-4 days +11:31:17.835640,14 days 09:31:17.777838,7 days 09:31:17.835640,-11 days +00:00:00,6 days 23:59:59.942198,2020-08-21 12:28:42.164360,2020-08-18,2020-09-08 22:00:00.000000,2020-08-28 22:00:00,NaT,before,2020-08-14 13:05:51.517542,Postpone Cash Request 12484,nice,2020-08-28 22:00:00,2020-09-10 22:00:00,nice
9,10,12484,5132,526,2020-08-14 12:28:42.222162,2020-08-25 16:19:27.021491,90.0,5.0,money_back,cancelled,instant,postpone,3 days 11:31:17.777838,-4 days +11:31:17.835640,14 days 09:31:17.777838,7 days 09:31:17.835640,-11 days +00:00:00,6 days 23:59:59.942198,2020-08-21 12:28:42.164360,2020-08-18,2020-09-08 22:00:00.000000,2020-08-28 22:00:00,NaT,before,2020-08-14 13:05:51.517542,Postpone Cash Request 12484,nice,2020-08-28 22:00:00,2020-09-10 22:00:00,nice


User ID: 12934


,index,id_cr,id_fe,user_id,created_at,created_at_fe,amount,fee,stat_cr,stat_fe,transfer_type,type,to_receive_ini,to_receive_bank,to_reimbur,to_reimbur_cash,to_end,to_send,send_at,cr_received_date,money_back_date,reimbursement_date,paid_at,charge_moment,moderated_at,reason,category,from_date,to_date,recovery_status
0,0,10094,2500,12934,2020-07-30 10:29:06.883916,2020-08-01 09:43:39.183502,50.0,5.0,money_back,accepted,instant,postpone,1 days 13:30:53.116084,-6 days +13:30:53.116778,16 days 11:30:53.116084,9 days 11:30:53.116778,-1 days +00:00:00,6 days 23:59:59.999306,2020-08-06 10:29:06.883222,2020-08-01,2020-08-16 22:00:00.000000,2020-08-15 22:00:00.000000,2020-08-01 09:43:43.408259,before,2020-07-30 10:45:55.890861,Postpone Cash Request 10094,nice,2020-08-13 10:29:25.986,2020-08-15 22:00:00,nice
1,1,10094,2374,12934,2020-07-30 10:29:06.883916,2020-07-30 10:45:56.972019,50.0,5.0,money_back,accepted,instant,instant_payment,1 days 13:30:53.116084,-6 days +13:30:53.116778,16 days 11:30:53.116084,9 days 11:30:53.116778,-1 days +00:00:00,6 days 23:59:59.999306,2020-08-06 10:29:06.883222,2020-08-01,2020-08-16 22:00:00.000000,2020-08-15 22:00:00.000000,2020-08-18 04:20:03.631389,after,2020-07-30 10:45:55.890861,Instant Payment Cash Request 10094,nice,NaT,NaT,nice
2,2,12969,4261,12934,2020-08-18 17:03:16.380648,2020-08-18 17:04:24.513048,50.0,5.0,money_back,accepted,instant,instant_payment,0 days 06:56:43.619352,-7 days +06:56:43.640985,14 days 00:00:13.055352,7 days 00:00:13.076985,-1 days +19:03:29.436000,6 days 23:59:59.978367,2020-08-25 17:03:16.359015,2020-08-19,2020-09-01 22:00:00.000000,2020-09-01 17:03:29.436000,2020-09-03 06:05:37.049392,after,2020-08-18 17:04:23.114327,Instant Payment Cash Request 12969,nice,NaT,NaT,nice
3,3,14683,6204,12934,2020-09-05 11:15:19.626216,2020-09-05 11:18:13.557782,100.0,5.0,money_back,accepted,instant,instant_payment,2 days 12:44:40.373784,-5 days +12:44:40.393945,14 days 00:00:14.434784,7 days 00:00:14.454945,0 days 10:38:36.069102,6 days 23:59:59.979839,2020-09-12 11:15:19.606055,2020-09-08,2020-09-19 00:36:57.991898,2020-09-19 11:15:34.061000,2020-09-19 00:36:58.048016,after,2020-09-05 11:18:12.405354,Instant Payment Cash Request 14683,nice,NaT,NaT,nice
4,4,17478,8857,12934,2020-09-23 13:44:15.487683,2020-09-23 13:44:54.658790,50.0,5.0,money_back,accepted,instant,instant_payment,0 days 10:15:44.512317,-14 days +10:15:44.534063,14 days 00:00:12.044317,0 days 00:00:12.066063,12 days 03:49:18.007404,13 days 23:59:59.978254,2020-10-07 13:44:15.465937,2020-09-24,2020-09-25 09:55:09.524596,2020-10-07 13:44:27.532000,2020-09-25 09:55:09.562910,after,2020-09-23 13:44:53.535180,Instant Payment Cash Request 17478,nice,NaT,NaT,nice
5,5,17791,11369,12934,2020-09-25 09:58:04.442154,2020-10-06 14:39:28.478601,10.0,5.0,money_back,accepted,instant,postpone,0 days 14:01:55.557846,-7 days +14:01:55.579246,39 days 13:01:55.557846,32 days 13:01:55.579246,28 days 08:17:12.508463,6 days 23:59:59.978600,2020-10-02 09:58:04.420754,2020-09-26,2020-10-06 14:42:47.491537,2020-11-03 23:00:00.000000,2020-10-06 14:39:33.571179,before,2020-09-25 09:58:39.706787,Postpone Cash Request 17791,nice,2020-10-09 09:58:18.793,2020-11-03 23:00:00,nice
6,6,17791,9212,12934,2020-09-25 09:58:04.442154,2020-09-25 09:58:41.146192,10.0,5.0,money_back,accepted,instant,instant_payment,0 days 14:01:55.557846,-7 days +14:01:55.579246,39 days 13:01:55.557846,32 days 13:01:55.579246,28 days 08:17:12.508463,6 days 23:59:59.978600,2020-10-02 09:58:04.420754,2020-09-26,2020-10-06 14:42:47.491537,2020-11-03 23:00:00.000000,2020-10-06 14:42:47.523911,after,2020-09-25 09:58:39.706787,Instant Payment Cash Request 17791,nice,NaT,NaT,nice
7,7,20137,12237,12934,2020-10-09 12:38:14.370122,2020-10-09 12:39:53.922936,10.0,5.0,money_back,accepted,instant,instant_payment,0 days 11:21:45.629878,0 days 11:21:21.266119,10 days 00:00:24.363759,10 days 00:00:00,8 days 19:29:13.165421,0 days 00:00:24.363759,2020-10-09 12:38:38.733881,2020-10-10,2020-10-10 17:09:25